In [11]:

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
from joblib import dump
import optuna
from optuna.integration import LightGBMPruningCallback
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import optuna
from optuna.integration import LightGBMPruningCallback
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np
import joblib
import optuna.visualization as vis
from sklearn.model_selection import cross_val_score, KFold
import warnings
warnings.filterwarnings("ignore")

In [12]:
# データの読み込み
train_data = pd.read_csv('../data_processed/train_data_processed.csv')
test_data = pd.read_csv('../data_processed/test_data_processed.csv')
submit_data = pd.read_csv('../data/test.csv')

X = train_data.drop(['attendance'], axis=1)
y = train_data['attendance']

# 学習データと評価データに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 評価データをDMatrixに変換
dtest = xgb.DMatrix(X_test)

In [13]:
# モデルを読み込む
XGBoost = xgb.Booster()
XGBoost.load_model('../models/XGBoost(交差検証法)')

In [15]:
# 評価データで予測
predictions = XGBoost.predict(dtest)

# RMSEを計算
rmse = mean_squared_error(y_test, predictions, squared=False)
print(f"RMSE: {rmse}")

# 予測値とy_testの値をデータフレームに変換
results = pd.DataFrame({'Prediction': predictions, 'Actual': y_test})

# 差分を計算
results['Difference'] = results['Actual'] - results['Prediction']

# 結果を出力

print(results)
# 評価用データをDMatrixに変換
dsubmit = xgb.DMatrix(test_data)

# 評価用データで予測
test_predictions = XGBoost.predict(dsubmit)

# 提出用データフレームを作成
submission = pd.DataFrame({
    'id': submit_data['id'],
    'attendance': test_predictions
})



RMSE: 1340.5699117758693
        Prediction   Actual   Difference
1116  10743.515625  10211.0  -532.515625
2358   8578.427734   8499.0   -79.427734
807   18519.781250  18171.0  -348.781250
2700  19560.917969  19977.0   416.082031
196   11570.718750   9142.0 -2428.718750
...            ...      ...          ...
2929  11919.902344  12144.0   224.097656
1211  17375.158203  15071.0 -2304.158203
3059  18385.146484  18219.0  -166.146484
1210  14524.304688  14486.0   -38.304688
2493  20875.826172  21132.0   256.173828

[674 rows x 3 columns]


In [ ]:
# ヘッダーを含まずにCSVファイルとして保存（'Deloitte Analytics/submission'フォルダに保存）
submission.to_csv('../submission/18XGBoost(交差検証法).csv', index=False, header=False)